In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [23]:
ithaca_data = pd.read_csv('datasets/hourly_ithaca_weather.csv').dropna()

In [24]:
ithaca_data.head()

,datetime (UTC),"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),relative_humidity (0-1),wind_speed (m/s),total_cloud_cover (0-1),total_precipitation (mm of water equivalent)
0,2005-01-01 00:00:00,"(42.439604, -76.496802)",era5,368.22,-5.0,9.63,6.49,0.81,5.00,0.83,0.01
1,2005-01-01 01:00:00,"(42.439604, -76.496802)",era5,368.22,-5.0,9.47,6.63,0.82,4.98,0.93,0.01
2,2005-01-01 02:00:00,"(42.439604, -76.496802)",era5,368.22,-5.0,9.59,6.76,0.83,5.28,0.90,0.00
3,2005-01-01 03:00:00,"(42.439604, -76.496802)",era5,368.22,-5.0,9.57,6.17,0.79,5.53,0.76,0.00
4,2005-01-01 04:00:00,"(42.439604, -76.496802)",era5,368.22,-5.0,9.52,6.47,0.81,5.44,0.57,0.00


In [25]:
ithaca_data = ithaca_data.rename(columns={'datetime (UTC)': 'datetime',
                                          'temperature (degC)': 'temperature',
                                          'dewpoint_temperature (degC)': 'dewpoint_temperature',
                                          'relative_humidity (0-1)': 'relative_humidity',
                                          'total_cloud_cover (0-1)': 'cloud_cover',
                                          'total_precipitation (mm of water equivalent)': 'precipitation'
                                          })

In [26]:
# Assuming 'datetime' column is in datetime format
ithaca_data['datetime'] = pd.to_datetime(ithaca_data['datetime'])

# Extract date from datetime and set it as index
ithaca_data['date'] = ithaca_data['datetime'].dt.date
ithaca_data.set_index('date', inplace=True)

# Define aggregation functions for each column
agg_functions = {'temperature': 'mean',
                 'dewpoint_temperature': 'mean',
                 'relative_humidity': 'mean',
                 'cloud_cover': 'mean',
                 'precipitation': 'sum'}

# Group by date and apply aggregation functions
daily_ithaca_data = ithaca_data.groupby('date').agg(agg_functions)

# Reset the index to get 'date' back as a column
daily_ithaca_data.reset_index(inplace=True)


daily_ithaca_data


,date,temperature,dewpoint_temperature,relative_humidity,cloud_cover,precipitation
0,2005-01-01,5.793333,1.602917,0.747083,0.633333,0.69
1,2005-01-02,0.222500,-2.469583,0.821667,0.902500,1.98
2,2005-01-03,4.257500,3.196667,0.927500,0.997500,5.18
3,2005-01-04,2.041250,1.375833,0.953333,0.997917,10.04
4,2005-01-05,-0.984583,-3.369167,0.839583,1.000000,3.77
...,...,...,...,...,...,...
6569,2022-12-27,-6.364167,-9.941250,0.758750,0.853750,0.36
6570,2022-12-28,-1.111250,-5.182917,0.752083,0.885417,0.00
6571,2022-12-29,3.182917,-2.744167,0.659583,0.574167,0.00
6572,2022-12-30,8.702917,3.053333,0.682500,0.806250,0.10


In [27]:
daily_ithaca_data.precipitation.describe()

count    6574.000000
mean        3.192536
std         6.213442
min         0.000000
25%         0.020000
50%         0.710000
75%         3.390000
max        96.420000
Name: precipitation, dtype: float64

In [32]:
# round down to 0 if precipitation is less than 1 mm
daily_ithaca_data['precipitation'] = daily_ithaca_data['precipitation'].apply(lambda x: max(0, 0 if x < 3 else x))


In [33]:
daily_ithaca_data.precipitation.value_counts()

0.00     4798
3.30        5
3.37        5
4.37        4
3.67        4
         ... 
4.53        1
20.43       1
22.59       1
19.73       1
5.66        1
Name: precipitation, Length: 1475, dtype: int64

In [34]:
daily_ithaca_data['rain_tomorrow'] = (daily_ithaca_data['precipitation'].shift(-1) > 0).astype(int)

In [35]:
daily_ithaca_data.to_csv('datasets/daily_ithaca_data.csv', index=False)